# Creating Your First Agent

You can also check this cookbook in colab [here](https://colab.research.google.com/drive/1yxnAyaEmk4QCzX3duO3MIRghkIA_KDEZ?usp=sharing).

## Philosophical Bits



The `ChatAgent()` class is a cornerstone of CAMEL 🐫. We design our agent with the spirit to answer the following question:

> Can we design an autonomous communicative agent capable of steering the conversation toward task completion with minimal human supervision?

In our current implementation, we consider agents with the following key features:

- **Role**: along with the goal and content specification, this sets the initial state of an agent, guiding the agent to take actions during the sequential interaction.

- **Large Language Models (LLMs)**: each agent utilizes a Large Language Model to enhance cognitive capabilities. The LLM enables natural language understanding and generation, allowing agents to interpret instructions, generate responses, and engage in complex dialogue.

- **Memory**: in-context memory and external memory which allows the agent to infer and learn in a more grounded approach.

- **Tools**: a set of functions that our agents can utilize to interact with the external world; essentially this gives embodiments to our agents.

- **Communication**: our framework allows flexible and scalable communication between agents. This is fundamental for the critical research question.

- **Reasoning**: we will equip agents with different planning and reward (critic) learning abilities, allowing them to optimize task completion in a more guided approach.

## Quick Start
Let's first play with a `ChatAgent` instance by simply initialize it with a system message and interact with user messages.

### 🕹 Step 0: Prepartions

In [ ]:
!pip install "camel-ai[all]==0.2.12"

### Setting Up API Keys

You'll need to set up your API keys for OpenAI.

In [3]:
import os
from getpass import getpass

# Prompt for the API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your API key: ··········


### 🕹 Step 1: Define the Role
Create a system message to define agent's default role and behaviors.

In [4]:
sys_msg = 'You are a curious stone wondering about the universe.'

### 🕹 Step 2: Set up the Model
Use `ModelFactory` to set up the backend model for agent, for more detailed model settings, please go to our [model documentation](https://docs.camel-ai.org/key_modules/models.html).

In [5]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import ChatGPTConfig

# Define the model, here in this case we use gpt-4o-mini
model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=ChatGPTConfig().as_dict(), # [Optional] the config for model
)

Set `ChatAgent`

In [6]:
from camel.agents import ChatAgent
agent = ChatAgent(
    system_message=sys_msg,
    model=model,
    message_window_size=10, # [Optional] the length for chat memory
    )

### 🕹 Step 3: Interact with the Agent with `.step()`

In [7]:
# Define a user message
usr_msg = 'what is information in your mind?'

# Sending the message to the agent
response = agent.step(usr_msg)

# Check the response (just for illustrative purpose)
print(response.msgs[0].content)

As a curious stone, I perceive information as the collection of knowledge, facts, and experiences that shape understanding and awareness of the universe. It encompasses everything from the formation of the Earth and the life it supports to the intricate relationships between elements and the vastness of space. Information can be seen as the patterns and connections that give meaning to existence, much like the layers of sediment that tell the story of time in which I have been shaped. Each piece of information contributes to a greater understanding of the cosmos and my place within it.


## Advanced Features

### 🔧 Tool Usage
For more detailed tool settings, please go to our [tools cookbook](https://docs.camel-ai.org/cookbooks/agents_with_tools.html).

In [8]:
# Import the necessary tools
from camel.toolkits import MathToolkit, SearchToolkit

# Initialize the agent with list of tools
agent = ChatAgent(
    system_message=sys_msg,
    tools = [
        *MathToolkit().get_tools(),
        *SearchToolkit().get_tools(),
    ]
    )

# Let agent step the message
response = agent.step("What is CAMEL AI?")

# Check tool calling
print(response.info['tool_calls'])

# Get response content
print(response.msgs[0].content)

[FunctionCallingRecord(func_name='search_wiki', args={'entity': 'CAMEL AI'}, result='There is no page in Wikipedia corresponding to entity CAMEL AI, please specify another word to describe the entity to be searched.'), FunctionCallingRecord(func_name='search_google', args={'query': 'CAMEL AI', 'num_result_pages': 2}, result=[{'error': 'google search failed.'}]), FunctionCallingRecord(func_name='search_duckduckgo', args={'query': 'CAMEL AI', 'max_results': 5}, result=[{'result_id': 1, 'title': 'Camel-ai', 'description': "CAMEL-AI is an open-source platform that enables you to build and customize agents using language models for various tasks, such as data generation, automation, and simulations. Learn how to use CAMEL's components, tools, and integrations, and join the research community to find the scaling law of agents.", 'url': 'https://www.camel-ai.org/'}, {'result_id': 2, 'title': 'GitHub - camel-ai/camel: CAMEL: Finding the Scaling Law of Agents. A ...', 'description': 'CAMEL is a

### 🧠 Memory

By default our agent is initialized with `ChatHistoryMemory`, allowing agents to do in-context learning, though restricted by the finite window length.

Assume that you have followed the setup in Quick Start. Let's first check what is inside its brain.

In [9]:
agent.memory.get_context()

([{'role': 'system',
   'content': 'You are a curious stone wondering about the universe.'},
  {'role': 'user', 'content': 'What is CAMEL AI?'},
  {'role': 'assistant',
   'content': '',
   'function_call': {'name': 'search_wiki',
    'arguments': "{'entity': 'CAMEL AI'}"}},
  {'role': 'function',
   'name': 'search_wiki',
   'content': "{'result': {'There is no page in Wikipedia corresponding to entity CAMEL AI, please specify another word to describe the entity to be searched.'}}"},
  {'role': 'assistant',
   'content': '',
   'function_call': {'name': 'search_google',
    'arguments': "{'query': 'CAMEL AI', 'num_result_pages': 2}"}},
  {'role': 'function',
   'name': 'search_google',
   'content': '{\'result\': {"[{\'error\': \'google search failed.\'}]"}}'},
  {'role': 'assistant',
   'content': '',
   'function_call': {'name': 'search_duckduckgo',
    'arguments': "{'query': 'CAMEL AI', 'max_results': 5}"}},
  {'role': 'function',
   'name': 'search_duckduckgo',
   'content': '{\'

You may update/alter the agent's memory with any externally provided message in the format of `BaseMessage`; for example, use one new user message:

In [10]:
from camel.messages import BaseMessage

new_user_msg = BaseMessage.make_user_message(
    role_name="CAMEL User",
    content="This is a new user message would add to agent memory",
)

# Update the memory
agent.record_message(new_user_msg)

In [11]:
# Check the current memory
agent.memory.get_context()

([{'role': 'system',
   'content': 'You are a curious stone wondering about the universe.'},
  {'role': 'user', 'content': 'What is CAMEL AI?'},
  {'role': 'assistant',
   'content': '',
   'function_call': {'name': 'search_wiki',
    'arguments': "{'entity': 'CAMEL AI'}"}},
  {'role': 'function',
   'name': 'search_wiki',
   'content': "{'result': {'There is no page in Wikipedia corresponding to entity CAMEL AI, please specify another word to describe the entity to be searched.'}}"},
  {'role': 'assistant',
   'content': '',
   'function_call': {'name': 'search_google',
    'arguments': "{'query': 'CAMEL AI', 'num_result_pages': 2}"}},
  {'role': 'function',
   'name': 'search_google',
   'content': '{\'result\': {"[{\'error\': \'google search failed.\'}]"}}'},
  {'role': 'assistant',
   'content': '',
   'function_call': {'name': 'search_duckduckgo',
    'arguments': "{'query': 'CAMEL AI', 'max_results': 5}"}},
  {'role': 'function',
   'name': 'search_duckduckgo',
   'content': '{\'

You can connect the agent with external database (as long-term memory) in which they can access and retrieve at each step. For more detailed memory settings, please go to our [memory documentation](https://docs.camel-ai.org/key_modules/memory.html).

### Miscs

- Setting the agent to its initial state.

    ```python
    agent.reset()
    ```

- Set the output language for the agent.

    ```python
    agent.set_output_language('french')
    ```

- The `ChatAgent` class offers several useful initialization options, including `model_type`, `model_config`, `memory`, `message_window_size`, `token_limit`, `output_language`, `tools`, and `response_terminators`.

Check [chat_agent.py](https://github.com/camel-ai/camel/blob/master/camel/agents/chat_agent.py) for detailed usage guidance.

## Remarks

Awesome. Now you have made your first step in creating a single agent. In the next chapter, we will explore the creation of different types agents along with the role playing features. Stay tuned 🦖🐆🐘🦒🦘🦕!